In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

spark = SparkSession.builder.getOrCreate()

# Load tables
cmx = spark.table("my_database.extracts.customer_master_xref")
re = spark.table("my_database.extracts.receipt_extract")

# Filter customer_master_xref on org
cmx_filtered = cmx.filter(col("org") == "ERP_LN")

# Join: right join to get all from receipt_extract
joined = cmx_filtered.join(
    re,
    (cmx_filtered.legacy_site_id == re.customer_site) &
    (cmx_filtered.legacy_location_id == re.customer_location),
    how="right"
)
joined.show()


In [0]:

result = joined.select(
    cmx.org, 
    cmx.customer_name, 
    cmx.fusion_location_id,
    cmx.fusion_site_id,
    re.receipt_number,
    re.receipt_amount,
    re.date   
)
result.show()

In [0]:

# Write to new Delta table
result.write.format("delta").mode("overwrite").saveAsTable("my_database.my_schema.Int_AR_receipt_integrate")